In [1]:
from bs4 import BeautifulSoup
import requests
import textwrap 
import time
import re
import sys
import csv
import pandas as pd 
import numpy as np

In [2]:
def Get_soup_bs4(url):
    requete = requests.get(url)
    page = requete.content
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [3]:
def find_between(s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
def find_after(s, first ):
    try:
        start = s.index( first ) + len( first )
        #end = s.index( last, start )
        return s[start:len(s)]
    except ValueError:
        return ""

In [5]:
urls = ["https://www.maison-retraite-selection.fr/maison-de-retraite/ehpad/4738-le-cercle-des-aines-de-saint-germain.html&pc=1"]

In [6]:

TYPE_ETABLISSEMENT =  re.compile(r"\bType d'établissement\b")
SOULUTION_ACCUEIL =  re.compile(r"\bSolutions d'accueil\b")
REMOVE_multiline = re.compile(r'\n(\s*\n)', re.MULTILINE)
data = []
data_init = []
data_init12 = []
data_init13 = []
data_init14 = []
data_init15 = []
for url in urls:
  soup_ = Get_soup_bs4(url)
  etablissement = [soup_.find('h1').text]
  print('Établissement : ', etablissement)

  adresse = soup_.find("div", {"class":"Hero__location"})
  adresse = REMOVE_multiline.sub(' ', adresse.text)
  adresse = [" ".join(adresse.split())]
  print('adresse : ', adresse)

  tepy_etabl = soup_.find("div", {"class":"row margin-top-3"}).text
  #tepy_etabl = REMOVE_multiline.sub(' ', tepy_etabl)
  str_list = list(filter(None, tepy_etabl.split('\n')))
  str_list12 = [x.split()[-1] for x in str_list]
  print('type d établissement et Statut : ', str_list12)

  #tarifs = soup_.find("p", {"class":"color-gray-dark margin-0"}).text 
  tarifs = soup_.find("div", {"class":'row d-flex align-items-center'}).text
  tarifs = REMOVE_multiline.sub(' ', tarifs)
  tarifs = re.sub('\s\s+', ' ', tarifs)
  tarifs = [find_between(tarifs, "de", "€")]
  tarifs = [tarifs[0] + '€']
  print("Trafis : ", tarifs)

  description = soup_.find("div", {'Tab__pane is-active'})
  description = REMOVE_multiline.sub(' ', description.text)
  lines = textwrap.wrap(description, 100, break_long_words=False)
  lines = '\n'.join(lines)
  print('description : ', lines)


  soulution_dacc = soup_.find('h2').text 
  soul_dacc = soup_.find("p", {"margin-left-md-3 margin-bottom-0"}).text
  soul_dacc = find_after(soul_dacc, 'Type de pathologie :')
  print('Solutions d\'accueil : ',  soul_dacc.split())


  plus_info = soup_.find("div", {"Host-solutions__inner margin-top-md-3"})
  plus_info = REMOVE_multiline.sub(' ', plus_info.text)
  plus_info = re.sub('\s\s+', ' ', plus_info)
  plus_info = [x for x in plus_info.split('Lire la suite') if x] 

  print(len(plus_info), plus_info)

  
  Espaces_confort = soup_.find_all("li", {"Comforts__item"})
  for item in Espaces_confort:
    text = item.text
    text = REMOVE_multiline.sub(' ', text)
    text = re.sub('\s\s+', ' ', text)
    data_init.append(text)
  print ('Espaces_confort : ', data_init)



  for ultag in soup_.find_all("ul", {"Social-lives__items margin-top-3"}):
    print(ultag)
    #for litag in ultag.find_all('li'):
     # print(litag)
  '''
  for ultag in soup_.find_all("ul", {"Social-lives__items margin-top-3"}):
    for litag in ultag.find_all('li'):

      litag = REMOVE_multiline.sub(' ', litag.text)
      litag = re.sub('\s\s+', ' ', litag)
      data_init12.append(litag)
  
  print('1 : ',data_init12)
  
  for ultag in soup_.find_all("section", {"Feature Care-team"}):
    #ultag = REMOVE_multiline.sub(' ', ultag.text)
    #ultag = re.sub('\s\s+', ' ', ultag)
    data_init13.append(ultag.text)
  
  print('2 : ', data_init13)

  for ultag in soup_.find_all("section", {"Feature Prices"}):
    #ultag = REMOVE_multiline.sub(' ', ultag.text)
    #ultag = re.sub('\s\s+', ' ', ultag)
    data_init14.append(ultag.text)
  
  print('3 : ', ultag.text)

  for ultag in soup_.find_all("section", {"Feature Accomodations"}):
    #ultag = REMOVE_multiline.sub(' ', ultag.text)
    #ultag = re.sub('\s\s+', ' ', ultag)
    data_init15.append(ultag.text)
  
  print('4 : ', data_init15)
  '''


  #print(data_init12)
  #data.append((title.text, adresse, tepy_etabl, tarifs, description, soul_dacc.text, plus_info, data_init))


Établissement :  ['LE CERCLE DES AÎNES DE SAINT-GERMAIN']
adresse :  ['58, route de Honfleur , 27500 SAINT-GERMAIN-VILLAGE']
type d établissement et Statut :  ['EHPAD', 'Privé']
Trafis :  [' 2914 €']
description :   Cadre et prestations. Aucun.  Installé en périphérie de Pont-Audemer, "La Venise Normande" dans une
zone verte et tranquille, cet EHPAD a été repris depuis peu par un petit groupe privé.  Son bâtiment
moderne mais sans charme particulier, a été totalement relooké et il offre à présent un intérieur
chaud, confortable et très plaisant. Les lieux de vie sont multiples et axés sur le bien-être: salle
de restaurant, d’animations, de coiffure, salons divers, bar, balnéo et espace snoezelen. Leur
décoration raffinée est aussi une véritable réussite. Composé de deux ailes et d’un bâtiment
central, il accueille sur trois niveaux, quatre-vingts résidents de toutes dépendances dans de très
bonnes conditions de confort. Les chambres sont toutes individuelles équipées de douche privativ

In [8]:
from selenium import webdriver


In [21]:
def download_page(url):
    browser = webdriver.Chrome()
    browser.get(url)
    return browser

In [23]:
driver = download_page(urls[0])


uls = driver.find("ul", class_="Social-lives__items margin-top-3")  
print(uls)

'''
all_li = driver.find_elements_by_tag_name("li")
for li in all_li:
    text = li.text
    print (text)
 '''   
driver.quit()

AttributeError: 'WebDriver' object has no attribute 'find'